In [11]:
#!/usr/bin/env python
# coding: utf-8

# load a bunch of stuff
from __future__ import division
# load
import cantera as ct
import numpy as np
import scipy
import pylab
import matplotlib
import matplotlib.pyplot  as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import cm
from matplotlib.ticker import NullFormatter, MaxNLocator, LogLocator
plt.switch_backend('agg')  # needed for saving figures
import csv
from pydas.dassl import DASSL
import os
import rmgpy
import rmg
import re
import operator
import pandas as pd
import pylab
from cycler import cycler
import seaborn as sns
import os
import multiprocessing
import itertools

In [12]:
# this chemkin file is from the cti generated by rmg
gas = ct.Solution('./rh/cantera/chem_annotated.cti', 'gas')
surf = ct.Interface('./rh/cantera/chem_annotated.cti', 'surface1', [gas])

gas_cov = ct.Solution('./rh_cov/cantera/chem_annotated.cti', 'gas')
surf_cov = ct.Interface('./rh_cov/cantera/chem_annotated.cti', 'surface1', [gas_cov])

print("This mechanism contains {} gas reactions and {} surface reactions".format(gas.n_reactions, surf.n_reactions))


This mechanism contains 40 gas reactions and 122 surface reactions


In [13]:
i_ar = gas.species_index('Ar')
i_ch4 = gas.species_index('CH4(2)')
i_o2 = gas.species_index('O2(3)')
i_co2 = gas.species_index('CO2(4)')
i_h2o = gas.species_index('H2O(5)')
i_h2 = gas.species_index('H2(6)')
i_co = gas.species_index('CO(7)')

# unit conversion factors to SI
mm = 0.001
cm = 0.01
ms = mm
minute = 60.0

#######################################################################
# Input Parameters
#######################################################################
t_in = 700  # K - in the paper, it was ~698.15K at the start of the cat surface and ~373.15 for the gas inlet temp
t_cat = t_in
length = 70 * mm  # Reactor length- m
diam = 16.5*mm  # Reactor diameter - in m
area = (diam/2.0)**2*np.pi  # Reactor cross section area (area of tube) in m^2
porosity = 0.81  # Monolith channel porosity, from Horn ref 17 sec 2.2.2
cat_area_per_vol = 16000  # I made this up, in m-1. 4500 is lowest that "work" for all base
flow_rate = 4.7  # slpm
flow_rate = flow_rate*.001/60  # m^3/s
tot_flow = 0.208  # from Horn 2007, constant inlet flow rate in mol/min, equivalent to 4.7 slpm
velocity = flow_rate/area  # m/s
# The PFR will be simulated by a chain of 'NReactors' stirred reactors.
NReactors = 7001

on_catalyst = 1000  # catalyst length 10mm, but it doesn't say where.  let's guess at 1 cm?
off_catalyst = 2000
dt = 1.0

reactor_len = length/(NReactors-1)
rvol = area * reactor_len * porosity
# catalyst area in one reactor
cat_area = cat_area_per_vol * rvol

In [14]:
def plotZoom(a):
    gas_out, surf_out, gas_names, surf_names, dist_array, T_array = a

    fig, axs = plt.subplots(1, 2)
    axs[0].set_prop_cycle(cycler('color', ['m', 'g', 'b', 'y', 'c', 'r', 'k', 'g']))

    for i in range(len(gas_out[0, :])):
        if i != i_ar:
            if gas_out[:, i].max() > 5.e-3:
                #             print(gas_names[i])
                axs[0].plot(dist_array, gas_out[:, i], label=gas_names[i])
                species_name = gas_names[i]
                if species_name.endswith(')'):
                    if species_name[-3] == '(':
                        species_name = species_name[0:-3]
                    else:
                        species_name = species_name[0:-4]
                if species_name == "O2":
                    axs[0].annotate("O$_2$", fontsize=18, color='y',
                                    xy=(dist_array[1100], gas_out[:, i][1100] + gas_out[:, i][1100] / 100.0),
                                    va='bottom', ha='center')
                elif species_name == "CO2":
                    axs[0].annotate("CO$_2$", fontsize=18, color='c',
                                    xy=(dist_array[2400], gas_out[:, i][2400] + gas_out[:, i][2400] / 10.0), va='bottom',
                                    ha='center')
                elif species_name == "CO":
                    axs[0].annotate("CO", fontsize=18, color='g', xy=(dist_array[2100], gas_out[:, i][2100] + 0.001),
                                    va='bottom', ha='center')
                elif species_name == "H2":
                    axs[0].annotate("H$_2$", fontsize=18, color='k', xy=(dist_array[2200], gas_out[:, i][2200] - 0.001),
                                    va='top', ha='center')
                elif species_name == "CH4":
                    axs[0].annotate("CH$_4$", fontsize=18, color='b',
                                    xy=(dist_array[1100], gas_out[:, i][1100] + gas_out[:, i][1100] / 100.0),
                                    va='bottom', ha='center')
                elif species_name == "H2O":
                    axs[0].annotate("H$_2$O", fontsize=18, color='r',
                                    xy=(dist_array[2100], gas_out[:, i][2100] + gas_out[:, i][2100] / 40.0 + 0.001), va='bottom',
                                    ha='center')
                else:
                    axs[0].annotate(species_name, fontsize=18,
                                    xy=(dist_array[-1], gas_out[:, i][-1] + gas_out[:, i][-1] / 10.0), va='top',
                                    ha='center')
            else:
                axs[0].plot(0, 0)

    axs[1].set_prop_cycle(cycler('color', ['m', 'g', 'b', 'y', 'c', 'r', 'k', 'g']))
    ax2 = axs[0].twinx()
    ax2.plot(dist_array, T_array, label='temperature', color='r', linestyle=':')
    axs[0].set_prop_cycle(cycler('color', ['m', 'g', 'b', 'y', 'c', 'r', 'k', 'g']))

    axs[0].plot([dist_array[on_catalyst], dist_array[on_catalyst]], [0, 0.2], linestyle='--', color='xkcd:grey')
    axs[0].plot([dist_array[off_catalyst], dist_array[off_catalyst]], [0, 0.2], linestyle='--', color='xkcd:grey')
    axs[0].annotate("catalyst", fontsize=18, xy=(dist_array[on_catalyst], 0.175), va='bottom', ha='left')
    axs[1].plot([dist_array[on_catalyst], dist_array[on_catalyst]], [600.0, 2000], linestyle='--', color='xkcd:grey')
    axs[1].plot([dist_array[off_catalyst], dist_array[off_catalyst]], [600.0, 2000], linestyle='--', color='xkcd:grey')
    axs[1].annotate("catalyst", fontsize=18, xy=(dist_array[on_catalyst], 1800), va='bottom', ha='left')

    for item in (
            axs[0].get_xticklabels() + axs[0].get_yticklabels() + ax2.get_xticklabels() + ax2.get_yticklabels()):
        item.set_fontsize(18)

    axs[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=False, shadow=False, ncol=4)
    axs[0].set_ylim(0., 0.1)
    axs[1].set_ylim(600.0, 2000)
    axs[0].set_xlim(8, 25)
    axs[1].set_xlim(8, 25)
    axs[0].set_xlabel('Distance (mm)', fontsize=22)
    axs[1].set_xlabel('Distance (mm)', fontsize=22)  # axs[0,1].set_xlabel('time (s)'); axs[1,1].set_xlabel('time (s)')
    axs[0].set_ylabel('flow/ mol/min', fontsize=22)
    ax2.set_ylabel('Temperature (K)', fontsize=22)
    ax2.set_ylim(600, 2000)
    ax2.set_xlim(8, 25)
    fig.delaxes(axs[1])  # THIS DELETES THE EXTRA SUBPLOT!
    fig.set_figheight(6)
    fig.set_figwidth(24)

    for n in range(len(gas_names)):
        if gas_names[n] == 'CH4(2)':
            c_in = gas_out[0][n]
        if gas_names[n] == 'O2(3)':
            o_in = gas_out[0][n]
    ratio = c_in / (o_in * 2)
    ratio = round(ratio, 1)

    out_dir = 'figures'
    os.path.exists(out_dir) or os.makedirs(out_dir)


def plotSurf(a):
    gas_out, surf_out, gas_names, surf_names, dist_array, T_array = a

    fig, axs = plt.subplots(1, 2)
    axs[0].set_prop_cycle(cycler('color', ['m', 'g', 'b', 'y', 'c', 'r', 'k', 'g']))

    for i in range(len(gas_out[0, :])):
        if i != i_ar:
            if gas_out[:, i].max() > 5.e-3:
                axs[0].plot(dist_array, gas_out[:, i], label=gas_names[i])
                species_name = gas_names[i]
                if species_name.endswith(')'):
                    if species_name[-3] == '(':
                        species_name = species_name[0:-3]
                    else:
                        species_name = species_name[0:-4]
                if species_name == "O2":
                    axs[0].annotate("O$_2$", fontsize=18,
                                    xy=(dist_array[2200], gas_out[:, i][2200] + gas_out[:, i][2200] / 100.0),
                                    va='bottom', ha='center')
                elif species_name == "CO2":
                    axs[0].annotate("CO$_2$", fontsize=18,
                                    xy=(dist_array[2200], gas_out[:, i][2200] + gas_out[:, i][2200] / 10.0), va='top',
                                    ha='center')
                elif species_name == "CO":
                    axs[0].annotate("CO", fontsize=18, xy=(dist_array[2200], gas_out[:, i][2200] + 0.001),
                                    va='bottom', ha='center')
                elif species_name == "CH2O":
                    axs[0].annotate("CH$_2$O", fontsize=18, xy=(dist_array[2200], gas_out[:, i][2200] + 0.001),
                                    va='bottom', ha='center')
                elif species_name == "CH4":
                    axs[0].annotate("CH$_4$", fontsize=18,
                                    xy=(dist_array[2200], gas_out[:, i][2200] + gas_out[:, i][2200] / 100.0),
                                    va='bottom', ha='center')
                elif species_name == "H2O":
                    axs[0].annotate("H$_2$O", fontsize=18,
                                    xy=(dist_array[2200], gas_out[:, i][2200] + gas_out[:, i][2200] / 40.0), va='top',
                                    ha='center')
                else:
                    axs[0].annotate(species_name, fontsize=18,
                                    xy=(dist_array[-1], gas_out[:, i][-1] + gas_out[:, i][-1] / 10.0), va='top',
                                    ha='center')
            else:
                axs[0].plot(0, 0)

    axs[1].set_prop_cycle(cycler('color', ['m', 'g', 'b', 'y', 'c', 'r', 'k', 'g']))
    # Plot two temperatures (of gas-phase and surface vs only surface.)
    for i in range(len(surf_out[0, :])):
        if surf_out[:, i].max() > 5.e-3:
            axs[1].semilogy(dist_array, surf_out[:, i], label=surf_names[i])
    axs[0].set_prop_cycle(cycler('color', ['m', 'g', 'b', 'y', 'c', 'r', 'k', 'g']))

    axs[0].plot([dist_array[on_catalyst], dist_array[on_catalyst]], [0, 0.2], linestyle='--', color='xkcd:grey')
    axs[0].plot([dist_array[off_catalyst], dist_array[off_catalyst]], [0, 0.2], linestyle='--', color='xkcd:grey')
    axs[0].annotate("catalyst", fontsize=18, xy=(dist_array[on_catalyst], 0.175), va='bottom', ha='left')
    axs[1].plot([dist_array[on_catalyst], dist_array[on_catalyst]], [0, 1.2], linestyle='--', color='xkcd:grey')
    axs[1].plot([dist_array[off_catalyst], dist_array[off_catalyst]], [0, 1.2], linestyle='--', color='xkcd:grey')
    axs[1].annotate("catalyst", fontsize=18, xy=(dist_array[on_catalyst], 1.1), va='bottom', ha='left')

    for item in (
            axs[0].get_xticklabels() + axs[0].get_yticklabels() + axs[1].get_xticklabels() + axs[1].get_yticklabels()):
        item.set_fontsize(18)

    axs[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=False, shadow=False, ncol=2)
    axs[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=False, shadow=False, ncol=4)
    axs[0].set_ylim(0., 0.1)
    axs[1].set_ylim(1e-10, 1.2)
    axs[0].set_xlim(5, 25)
    axs[1].set_xlim(9, 21)
    axs[0].set_xlabel('Distance (mm)', fontsize=22)
    axs[1].set_xlabel('Distance (mm)', fontsize=22)
    axs[0].set_ylabel('flow/ mol/min', fontsize=22)
    axs[1].set_ylabel('Site fraction', fontsize=22)
    fig.delaxes(axs[0])  # THIS DELETES THE EXTRA SUBPLOT!

    fig.set_figheight(6)
    fig.set_figwidth(18)

    for n in range(len(gas_names)):
        if gas_names[n] == 'CH4(2)':
            c_in = gas_out[0][n]
        if gas_names[n] == 'O2(3)':
            o_in = gas_out[0][n]
    ratio = c_in / (o_in * 2)
    ratio = round(ratio, 1)

    out_dir = 'figures'
    os.path.exists(out_dir) or os.makedirs(out_dir)
    fig.clf()


def monolithFull(gas, surf, temp, mol_in, rtol=1.0e-12, atol=1.0e-18, verbose=False, sens=False):
    """
    Verbose prints out values as you go along
    Sens is for sensitivity, in the form [perturbation, reaction #]
    """
    ch4, o2, ar = mol_in
    ratio = ch4/(2*o2)
    ratio = round(ratio, 1)
    ch4 = str(ch4)
    o2 = str(o2)
    ar = str(ar)
    X = str('CH4(2):' + ch4 + ', O2(3):' + o2 + ', Ar:' + ar)
    gas.TPX = 273.15, ct.one_atm, X  # need to initialize mass flow rate at STP
    mass_flow_rate = flow_rate * gas.density_mass
    gas.TPX = temp, ct.one_atm, X
    temp_cat = temp
    surf.TP = temp_cat, ct.one_atm
    surf.coverages = 'X(1):1.0'
    gas.set_multiplier(1.0)

    TDY = gas.TDY
    cov = surf.coverages

    if verbose is True:
        print('  distance(mm)   X_CH4        X_O2        X_H2       X_CO       X_H2O       X_CO2')

    # create a new reactor
    gas.TDY = TDY
    r = ct.IdealGasReactor(gas)
    r.volume = rvol

    # create a reservoir to represent the reactor immediately upstream. Note
    # that the gas object is set already to the state of the upstream reactor
    upstream = ct.Reservoir(gas, name='upstream')

    # create a reservoir for the reactor to exhaust into. The composition of
    # this reservoir is irrelevant.
    downstream = ct.Reservoir(gas, name='downstream')

    # Add the reacting surface to the reactor. The area is set to the desired
    # catalyst area in the reactor.
    rsurf = ct.ReactorSurface(surf, r, A=cat_area)

    # The mass flow rate into the reactor will be fixed by using a
    # MassFlowController object.
    # mass_flow_rate = velocity * gas.density_mass * area  # kg/s
    # mass_flow_rate = flow_rate * gas.density_mass
    m = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)

    # We need an outlet to the downstream reservoir. This will determine the
    # pressure in the reactor. The value of K will only affect the transient
    # pressure difference.
    v = ct.PressureController(r, downstream, master=m, K=1e-5)

    sim = ct.ReactorNet([r])
    sim.max_err_test_fails = 12

    # set relative and absolute tolerances on the simulation
    sim.rtol = rtol
    sim.atol = atol

    gas_names = gas.species_names
    surf_names = surf.species_names
    gas_out = []
    surf_out = []
    dist_array = []
    T_array = []

    surf.set_multiplier(0.0)  # no surface reactions until the gauze
    for n in range(NReactors):
        # Set the state of the reservoir to match that of the previous reactor
        gas.TDY = r.thermo.TDY
        upstream.syncState()
        if n == on_catalyst:
            surf.set_multiplier(1.0)
            if sens is not False:
                surf.set_multiplier(1.0 + sens[0], sens[1])
        if n == off_catalyst:
            surf.set_multiplier(0.0)
        sim.reinitialize()
        sim.advance_to_steady_state()
        dist = n * reactor_len * 1.0e3  # distance in mm
        dist_array.append(dist)
        T_array.append(surf.T)
        # print "mass_flow_rate", mass_flow_rate,  v.mdot(sim.time), "kg/s"
        kmole_flow_rate = mass_flow_rate / gas.mean_molecular_weight  # kmol/s
        gas_out.append(1000 * 60 * kmole_flow_rate * gas.X.copy())  # molar flow rate in moles/minute
        surf_out.append(surf.X.copy())

        # make reaction diagrams
        out_dir = 'rxnpath'
        os.path.exists(out_dir) or os.makedirs(out_dir)
        elements = ['H', 'O']
        locations_of_interest = [1000, 1150, 1160, 1183, 1196, 1999]
        if sens is False:
            for l in locations_of_interest:
                if n == l:
                    location = str(n / 100)

                    diagram = ct.ReactionPathDiagram(surf, 'X')
                    diagram.title = 'rxn path'
                    diagram.label_threshold = 1e-9
                    dot_file = out_dir + '/rxnpath-' + str(ratio) + '-x-' + location + 'mm.dot'
                    img_file = out_dir + '/rxnpath-' + str(ratio) + '-x-' + location + 'mm.png'
                    img_path = os.path.join(out_dir, img_file)
                    diagram.write_dot(dot_file)
                    os.system('dot {0} -Tpng -o{1} -Gdpi=200'.format(dot_file, img_file))

                    for element in elements:
                        diagram = ct.ReactionPathDiagram(surf, element)
                        diagram.title = element + 'rxn path'
                        diagram.label_threshold = 1e-9
                        dot_file = out_dir + '/rxnpath-' + str(ratio) + '-surf-' + location + 'mm-' + element + '.dot'
                        img_file = out_dir + '/rxnpath-' + str(ratio) + '-surf-' + location + 'mm-' + element + '.png'
                        img_path = os.path.join(out_dir, img_file)
                        diagram.write_dot(dot_file)
                        os.system('dot {0} -Tpng -o{1} -Gdpi=200'.format(dot_file, img_file))
        else:
            pass

        if verbose is True:
            if not n % 100:
                print('  {0:10f}  {1:10f}  {2:10f}  {3:10f} {4:10f} {5:10f} {6:10f}'.format(dist, *gas[
                    'CH4(2)', 'O2(3)', 'H2(6)', 'CO(7)', 'H2O(5)', 'CO2(4)'].X * 1000 * 60 * kmole_flow_rate))

    gas_out = np.array(gas_out)
    surf_out = np.array(surf_out)
    gas_names = np.array(gas_names)
    surf_names = np.array(surf_names)
    data_out = gas_out, surf_out, gas_names, surf_names, dist_array, T_array
    return data_out

In [15]:
rtol = [1.0e-9, 1.0e-10, 1.0e-11, 1.0e-12, 1.0e-13, 1.0e-14]
atol = [1.0e-16, 1.0e-17, 1.0e-18, 1.0e-19, 1.0e-20, 1.0e-21, 1.0e-22, 1.0e-23, 1.0e-24]

tols = list(itertools.product(rtol,atol))


In [16]:
ratio = 1.0
fo2 = 1 / (2. * ratio + 1 + 79 / 21)
fch4 = 2 * fo2 * ratio
far = 79 * fo2 / 21
ratio_in = [fch4, fo2, far]

a = monolithFull(gas, surf, t_in, ratio_in, verbose=True, rtol=1.0e-9, atol=1.0e-16)
gas_out, surf_out, gas_names, surf_names, dist_array, T_array = a

# b = monolithFull(gas_cov, surf_cov, t_in, ratio_in, verbose=True)
# gas_out_cov, surf_out_cov, gas_names_cov, surf_names_cov, dist_array_cov, T_array_cov = b

  distance(mm)   X_CH4        X_O2        X_H2       X_CO       X_H2O       X_CO2
    0.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    1.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    2.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    3.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    4.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    5.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    6.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    7.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    8.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    9.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
   10.000000    0.062015    0.031003    0.000008   0.000000   0.000003   0.000006
   11.000000    

In [17]:
b = monolithFull(gas_cov, surf_cov, t_in, ratio_in, verbose=True, rtol=1.0e-9, atol=1.0e-16)
gas_out_cov, surf_out_cov, gas_names_cov, surf_names_cov, dist_array_cov, T_array_cov = b

  distance(mm)   X_CH4        X_O2        X_H2       X_CO       X_H2O       X_CO2
    0.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    1.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    2.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    3.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    4.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    5.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    6.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    7.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    8.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
    9.000000    0.062021    0.031011    0.000000   0.000000   0.000000   0.000000
   10.000000    0.062015    0.031003    0.000008   0.000000   0.000003   0.000006
   11.000000    

In [18]:
temp_profile = np.array([ # [reactor location (mm), temperature (K)]
        [0.,373.15],
        [1.,373.15],
        [2.,373.15],
        [3.,373.15],
        [4.,373.15],
        [5.,373.15],
        [6.,398.15],
        [7.,448.15],
        [8.,498.15],
        [9.,573.15],
        [10.,698.15],
        [11.,998.15],
        [12.,1173.15],
        [13.,1239.15],
        [14.,1256.15],
        [15.,1255.15],
        [16.,1248.15],
        [17.,1230.15],
        [18.,1213.15],
        [19.,1198.15],
        [20.,1188.15],
        [21.,1183.15],
        [22.,1178.15],
        [23.,1173.15],
        [24.,1173.15],
        [25.,1173.15],])

ch4_in = 21/71*.208
ch4_profile = np.array([
        [0.,ch4_in],
        [1.,ch4_in],
        [2.,ch4_in],
        [3.,ch4_in],
        [4.,ch4_in],
        [5.,ch4_in],
        [6.,ch4_in],
        [7.,ch4_in],
        [8.,ch4_in],
        [9.,ch4_in],
        [10.,ch4_in],
        [10.5,0.06],
        [11.,0.055],
        [11.5,0.0475],
        [12.,0.041],
        [12.25,0.036],
        [12.5,0.034],
        [13.,0.033],
        [13.5,0.0327],
        [14.,0.03],
        [14.5,0.028],
        [15.,0.0275],
        [15.5,0.025],
        [16.,0.024],
        [17.,0.023],
        [18.,0.023],
        [19.,0.023],
        [20.,0.023],
        [21.,0.022],
        [22.,0.021],
        [23.,0.021],
        [24.,0.02],
        [25.,0.02],])

o2_in = (1 / (2. + 1 + 79 / 21)) * .208
o2_profile = np.array([
        [0.,o2_in],
        [1.,o2_in],
        [2.,o2_in],
        [3.,o2_in],
        [4.,o2_in],
        [5.,o2_in],
        [6.,o2_in],
        [7.,o2_in],
        [8.,o2_in],
        [9.,o2_in],
        [10.,o2_in],
        [10.25,0.0306],
        [10.5,0.03],
        [11.,0.02],
        [11.5,0.01],
        [12.,0.005],
        [13.,0.001],
        [14.,0.],
        [15.,0.],
        [16.,0.],
        [17.,0.],
        [18.,0.],
        [19.,0.],
        [20.,0.],
        [21.,0.],
        [22.,0.],
        [23.,0.],
        [24.,0.],
        [25.,0.],])

h2_profile = np.array([
        [0.,0],
        [1.,0],
        [2.,0],
        [3.,0],
        [4.,0],
        [5.,0],
        [6.,0],
        [7.,0],
        [8.,0],
        [9.,0],
        [10.,0],
        [11.,0.0075],
        [12.,0.02],
        [13.,0.0325],
        [14.,0.0425],
        [15.,0.0525],
        [16.,0.0575],
        [17.,0.06],
        [18.,0.0625],
        [19.,0.0625],
        [20.,0.065],
        [21.,0.069],
        [22.,0.07],
        [23.,0.07],
        [24.,0.071],
        [25.,0.071],])

co_profile = np.array([
        [0.,0],
        [1.,0],
        [2.,0],
        [3.,0],
        [4.,0],
        [5.,0],
        [6.,0],
        [7.,0],
        [8.,0],
        [9.,0],
        [10.,0],
        [11.,0.0075],
        [12.,0.02],
        [13.,0.0275],
        [14.,0.03],
        [15.,0.0325],
        [16.,0.035],
        [17.,0.0375],
        [18.,0.0375],
        [19.,0.0375],
        [20.,0.0375],
        [21.,0.0375],
        [22.,0.038],
        [23.,0.038],
        [24.,0.04],
        [25.,0.04],])

h2o_profile = np.array([
        [0.,0],
        [1.,0],
        [2.,0],
        [3.,0],
        [4.,0],
        [5.,0],
        [6.,0],
        [7.,0],
        [8.,0],
        [9.,0],
        [10.,0],
        [11.,0.016],
        [12.,0.026],
        [12.5,0.03],
        [13.,0.026],
        [14.,0.022],
        [15.,0.02],
        [16.,0.019],
        [17.,0.019],
        [18.,0.019],
        [19.,0.016],
        [20.,0.015],
        [21.,0.013],
        [22.,0.013],
        [23.,0.013],
        [24.,0.013],
        [25.,0.015],])

co2_profile = np.array([
        [0.,0],
        [1.,0],
        [2.,0],
        [3.,0],
        [4.,0],
        [5.,0],
        [6.,0],
        [7.,0],
        [8.,0],
        [9.,0],
        [10.,0],
        [11.,0.001],
        [12.,0.001],
        [12.5,0.001],
        [13.,0.001],
        [14.,0.001],
        [15.,0.001],
        [16.,0.001],
        [17.,0.001],
        [18.,0.001],
        [19.,0.001],
        [20.,0.001],
        [21.,0.001],
        [22.,0.001],
        [23.,0.001],
        [24.,0.001],
        [25.,0.001],])

from scipy.interpolate import interp1d

dist = np.linspace(0,25,2501)
ch4 = interp1d(ch4_profile[:,0],ch4_profile[:,1],kind='cubic')
o2 = interp1d(o2_profile[:,0],o2_profile[:,1],kind='cubic')
h2 = interp1d(h2_profile[:,0],h2_profile[:,1],kind='cubic')
co = interp1d(co_profile[:,0],co_profile[:,1],kind='cubic')
h2o = interp1d(h2o_profile[:,0],h2o_profile[:,1],kind='cubic')
co2 = interp1d(co2_profile[:,0],co2_profile[:,1],kind='cubic')
temp = interp1d(temp_profile[:,0],temp_profile[:,1],kind='cubic')

# plt.plot(dist,ch4(dist),color='g')
# plt.annotate("CH$_4$", fontsize=16, color='g',
#             xy=(dist_array[700], .062),
#             va='bottom', ha='center')
# plt.plot(dist,o2(dist),color='orange')
# plt.annotate("O$_2$", fontsize=16, color='orange',
#             xy=(dist_array[700], .032),
#             va='bottom', ha='center')
# plt.plot(dist,h2(dist),color='k')
# plt.annotate("H$_2$", fontsize=16, color='k',
#             xy=(dist_array[2300], .072),
#             va='bottom', ha='center')
# plt.plot(dist,co(dist),color='limegreen')
# plt.annotate("CO", fontsize=16, color='limegreen',
#             xy=(dist_array[2300], .03),
#             va='bottom', ha='center')
# plt.plot(dist,co2(dist),color='blue')
# plt.annotate("CO$_2$", fontsize=16, color='blue',
#             xy=(dist_array[2200], .002),
#             va='bottom', ha='center')
# plt.plot(dist,h2o(dist),color='dodgerblue')
# plt.annotate("H$_2$O", fontsize=16, color='dodgerblue',
#             xy=(dist_array[1600], .01),
#             va='bottom', ha='center')
# plt.plot([dist_array[on_catalyst], dist_array[on_catalyst]], [0, 0.1], linestyle='--', color='xkcd:grey')
# plt.plot([dist_array[off_catalyst], dist_array[off_catalyst]], [0, 0.1], linestyle='--', color='xkcd:grey')
# plt.annotate("catalyst", fontsize=13, xy=(dist_array[1500], 0.095), va='center', ha='center')

# plt.ylabel('Flow (mol/min)', fontsize=16)
# plt.xlabel('Position (mm)', fontsize=16)
# plt.ylim((0,0.1))
# plt.xlim((5,25))
# plt.xticks(fontsize=13)
# plt.yticks(fontsize=13)
# plt.xticks(np.arange(6,26,2))

# ax2 = plt.twinx()
# ax2.plot(dist,temp(dist), ':', color='r')
# ax2.set_ylim(300, 2000)
# ax2.annotate("T",fontsize=16, color='r',
#              xy=(dist_array[2300], 1310),
#              va='top', ha='center')
# ax2.set_ylabel('Temperature (K)', fontsize=16)
# for item in (ax2.get_yticklabels()):
#              item.set_fontsize(13)
# plt.savefig('./paperplots/horn_dataPt.pdf',bbox_inches='tight',dpi=300)
# plt.show()



# # plotting ch4
# plt.plot(dist_array,gas_out[:,3], label=gas_names[3],color='g')
# plt.annotate("CH$_4$", fontsize=16, color='g',
#             xy=(dist_array[700], gas_out[:, 3][500] + gas_out[:, 3][500] / 100.0),
#             va='bottom', ha='center')
# plt.plot(dist,ch4(dist),'--',color='g')
# # plotting o2
# plt.plot(dist_array,gas_out[:,4], label=gas_names[4],color='orange')
# plt.annotate("O$_2$", fontsize=16, color='orange',
#             xy=(dist_array[700], gas_out[:, 4][500] + gas_out[:, 4][500] / 100.0),
#             va='bottom', ha='center')
# plt.plot(dist,o2(dist),'--',color='orange')

# plt.plot([dist_array[on_catalyst], dist_array[on_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
# plt.plot([dist_array[off_catalyst], dist_array[off_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
# plt.annotate("catalyst", fontsize=13, xy=(dist_array[1500], 0.095), va='center', ha='center')

# plt.ylabel('Flow (mol/min)', fontsize=16)
# plt.xlabel('Position (mm)', fontsize=16)
# plt.ylim((0,0.1))
# plt.xticks(fontsize=13)
# plt.yticks(fontsize=13)
# plt.xticks(np.arange(6,26,2))

# ax2 = plt.twinx()
# ax2.plot(dist,temp(dist), '--', color='r')
# ax2.plot(dist_array,T_array,color='r')
# ax2.annotate("T",fontsize=16, color='r',
#              xy=(dist_array[2300], T_array[2300] -150),
#              va='top', ha='center')
# ax2.set_ylim(300, 2000)
# ax2.set_ylabel('Temperature (K)', fontsize=16)
# for item in (ax2.get_yticklabels()):
#              item.set_fontsize(13)

# plt.xlim((5,25))  # cutting off where the horn paper cuts off the plots
# # plt.savefig('./paperplots/compare_ch4_o2_tempPt.pdf',bbox_inches='tight',dpi=300)
# # plt.show()


# # In[14]:


# # plotting h2
# plt.plot(dist_array,gas_out[:,7], label=gas_names[7],color='k')
# plt.annotate("H$_2$", fontsize=16, color='k',
#             xy=(dist_array[2200], gas_out[:, 7][2200] + gas_out[:, 7][2200] / 100.0),
#             va='bottom', ha='center')
# plt.plot(dist,h2(dist),'--',color='k')
# # plotting co
# plt.plot(dist_array,gas_out[:,8], label=gas_names[8],color='limegreen')
# plt.annotate("CO", fontsize=16, color='limegreen',
#             xy=(dist_array[1800], gas_out[:, 8][1800] + gas_out[:, 8][1800] / 100.0),
#             va='bottom', ha='center')
# plt.plot(dist,co(dist),'--',color='limegreen')

# plt.plot([dist_array[on_catalyst], dist_array[on_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
# plt.plot([dist_array[off_catalyst], dist_array[off_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
# plt.annotate("catalyst", fontsize=13, xy=(dist_array[1500], 0.095), va='center', ha='center')

# plt.ylabel('Flow (mol/min)', fontsize=16)
# plt.xlabel('Position (mm)', fontsize=16)
# plt.ylim((0,0.1))
# plt.xticks(fontsize=13)
# plt.yticks(fontsize=13)
# plt.xticks(np.arange(6,26,2))

# plt.xlim((5,25))  # cutting off where the horn paper cuts off the plots
# # plt.savefig('./paperplots/compare_h2_coPt.pdf',bbox_inches='tight',dpi=300)
# # plt.show()


# # In[15]:


# # plotting h2o
# plt.plot(dist_array,gas_out[:,6], label=gas_names[6],color='dodgerblue')
# plt.annotate("H$_2$O", fontsize=16, color='dodgerblue',
#             xy=(dist_array[2200], gas_out[:, 6][2200] -.01),
#             va='bottom', ha='center')
# plt.plot(dist,h2o(dist),'--',color='dodgerblue')
# # plotting co2
# plt.plot(dist_array,gas_out[:,5], label=gas_names[5],color='blue')
# plt.annotate("CO$_2$", fontsize=16, color='blue',
#             xy=(dist_array[1400], .009),
#             va='bottom', ha='center')
# plt.plot(dist,co2(dist),'--',color='blue')

# plt.plot([dist_array[on_catalyst], dist_array[on_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
# plt.plot([dist_array[off_catalyst], dist_array[off_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
# plt.annotate("catalyst", fontsize=13, xy=(dist_array[1500], 0.095), va='center', ha='center')

# plt.ylabel('Flow (mol/min)', fontsize=16)
# plt.xlabel('Position (mm)', fontsize=16)
# plt.ylim((0,0.1))
# plt.xticks(fontsize=13)
# plt.yticks(fontsize=13)
# plt.xticks(np.arange(6,26,2))

# plt.xlim((5,25))  # cutting off where the horn paper cuts off the plots
# # plt.savefig('./paperplots/compare_h2o_co2Pt.pdf',bbox_inches='tight',dpi=300)
# # plt.show()


# # In[16]:


# # plotting ch4
# plt.plot(dist_array,gas_out[:,3], label=gas_names[3],color='g')
# plt.annotate("CH$_4$", fontsize=16, color='g',
#             xy=(dist_array[700], gas_out[:, 3][500] + gas_out[:, 3][500] / 100.0),
#             va='bottom', ha='center')
# plt.plot(dist,ch4(dist),'--',color='g')
# # plotting o2
# plt.plot(dist_array,gas_out[:,4], label=gas_names[4],color='orange')
# plt.annotate("O$_2$", fontsize=16, color='orange',
#             xy=(dist_array[700], gas_out[:, 4][500] + gas_out[:, 4][500] / 100.0),
#             va='bottom', ha='center')
# plt.plot(dist,o2(dist),'--',color='orange')
# # plotting h2
# plt.plot(dist_array,gas_out[:,7], label=gas_names[7],color='k')
# plt.annotate("H$_2$", fontsize=16, color='k',
#             xy=(dist_array[2200], gas_out[:, 7][2200] + gas_out[:, 7][2200] / 100.0),
#             va='bottom', ha='center')
# plt.plot(dist,h2(dist),'--',color='k')
# # plotting co
# plt.plot(dist_array,gas_out[:,8], label=gas_names[8],color='limegreen')
# plt.annotate("CO", fontsize=16, color='limegreen',
#             xy=(dist_array[1800], gas_out[:, 8][1800] + gas_out[:, 8][1800] / 100.0),
#             va='bottom', ha='center')
# plt.plot(dist,co(dist),'--',color='limegreen')
# # plotting h2o
# plt.plot(dist_array,gas_out[:,6], label=gas_names[6],color='dodgerblue')
# plt.annotate("H$_2$O", fontsize=16, color='dodgerblue',
#             xy=(dist_array[2200], gas_out[:, 6][2200] -.01),
#             va='bottom', ha='center')
# plt.plot(dist,h2o(dist),'--',color='dodgerblue')
# # plotting co2
# plt.plot(dist_array,gas_out[:,5], label=gas_names[5],color='blue')
# plt.annotate("CO$_2$", fontsize=16, color='blue',
#             xy=(dist_array[1400], .009),
#             va='bottom', ha='center')
# plt.plot(dist,co2(dist),'--',color='blue')

# plt.plot([dist_array[on_catalyst], dist_array[on_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
# plt.plot([dist_array[off_catalyst], dist_array[off_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
# plt.annotate("catalyst", fontsize=13, xy=(dist_array[1500], 0.095), va='center', ha='center')

# plt.ylabel('Flow (mol/min)', fontsize=16)
# plt.xlabel('Position (mm)', fontsize=16)
# plt.ylim((0,0.1))
# plt.xticks(fontsize=13)
# plt.yticks(fontsize=13)
# plt.xticks(np.arange(6,26,2))

# # ax2 = plt.twinx()
# # ax2.plot(dist,temp(dist), '--', color='r')
# # ax2.plot(dist_array,T_array,color='r')
# # ax2.annotate("T",fontsize=16, color='r',
# #              xy=(dist_array[2300], T_array[2300] -150),
# #              va='top', ha='center')
# # ax2.set_ylim(300, 2000)
# # ax2.set_ylabel('Temperature (K)', fontsize=16)
# # for item in (ax2.get_yticklabels()):
# #              item.set_fontsize(13)

# plt.xlim((5,25))  # cutting off where the horn paper cuts off the plots
# # plt.savefig('./paperplots/all_comparePt.pdf',bbox_inches='tight',dpi=300)
# # plt.show()





# dist = np.linspace(0,25,2501)


# fig, axs = plt.subplots(3, 1)

# # plotting ch4
# axs[0].plot(dist_array,gas_out[:,3], label=gas_names[3],color='g')
# axs[0].annotate("CH$_4$", fontsize=16, color='g',
#             xy=(dist_array[700], gas_out[:, 3][500] + gas_out[:, 3][500] / 100.0),
#             va='bottom', ha='center')
# axs[0].plot(dist,ch4(dist),'--',color='g')
# # plotting o2
# axs[0].plot(dist_array_cov,gas_out_cov[:,4], label=gas_names_cov[4],color='orange')
# axs[0].annotate("O$_2$", fontsize=16, color='orange',
#             xy=(dist_array_cov[700], gas_out_cov[:, 4][500] + gas_out_cov[:, 4][500] / 100.0),
#             va='bottom', ha='center')
# axs[0].plot(dist,o2(dist),'--',color='orange')

# axs[0].plot([dist_array_cov[on_catalyst], dist_array_cov[on_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
# axs[0].plot([dist_array_cov[off_catalyst], dist_array_cov[off_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
# axs[0].annotate("catalyst", fontsize=13, xy=(dist_array[1500], 0.095), va='center', ha='center')

# axs[0].set_ylabel('Flow (mol/min)', fontsize=16)
# axs[0].set_ylim((0,0.1))
# for item in (axs[0].get_xticklabels()):
#              item.set_fontsize(13)
# for item in (axs[0].get_yticklabels()):
#              item.set_fontsize(13)
# axs[0].set_xticks(np.arange(6,26,2))

# ax2 = axs[0].twinx()
# ax2.plot(dist,temp(dist), '--', color='r')
# ax2.plot(dist_array_cov,T_array_cov,color='r')
# ax2.annotate("T (K)",fontsize=16, color='r',
#              xy=(dist_array_cov[2300], T_array_cov[2300] -150),
#              va='top', ha='center')
# ax2.set_ylim(300, 2000)
# for item in (ax2.get_yticklabels()):
#              item.set_fontsize(13)
# axs[0].set_xlim((5,25))

# # plotting h2
# axs[1].plot(dist_array_cov,gas_out_cov[:,7], label=gas_names_cov[7],color='k')
# axs[1].annotate("H$_2$", fontsize=16, color='k',
#             xy=(dist_array_cov[2200], 0.0575),
#             va='bottom', ha='center')
# axs[1].plot(dist,h2(dist),'--',color='k')
# # plotting co
# axs[1].plot(dist_array_cov,gas_out_cov[:,8], label=gas_names_cov[8],color='limegreen')
# axs[1].annotate("CO", fontsize=16, color='limegreen',
#             xy=(dist_array_cov[1800], gas_out_cov[:, 8][1800] + gas_out_cov[:, 8][1800] / 100.0 - 0.1),
#             va='bottom', ha='center')
# axs[1].plot(dist,co(dist),'--',color='limegreen')

# axs[1].plot([dist_array_cov[on_catalyst], dist_array_cov[on_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
# axs[1].plot([dist_array_cov[off_catalyst], dist_array_cov[off_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')

# axs[1].set_ylabel('Flow (mol/min)', fontsize=16)
# axs[1].set_ylim((0,0.1))
# for item in (axs[1].get_xticklabels()):
#              item.set_fontsize(13)
# for item in (axs[1].get_yticklabels()):
#              item.set_fontsize(13)
# axs[1].set_xticks(np.arange(6,26,2))

# axs[1].set_xlim((5,25))  # cutting off where the horn paper cuts off the plots

# # plotting h2o
# axs[2].plot(dist_array_cov,gas_out_cov[:,6], label=gas_names_cov[6],color='dodgerblue')
# axs[2].annotate("H$_2$O", fontsize=16, color='dodgerblue',
#             xy=(dist_array_cov[2200], 0.035),
#             va='bottom', ha='center')
# axs[2].plot(dist,h2o(dist),'--',color='dodgerblue')
# # plotting co2
# axs[2].plot(dist_array_cov,gas_out_cov[:,5], label=gas_names_cov[5],color='blue')
# axs[2].annotate("CO$_2$", fontsize=16, color='blue',
#             xy=(dist_array_cov[1400], .009),
#             va='bottom', ha='center')
# axs[2].plot(dist,co2(dist),'--',color='blue')

# axs[2].plot([dist_array_cov[on_catalyst], dist_array_cov[on_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
# axs[2].plot([dist_array_cov[off_catalyst], dist_array_cov[off_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')

# axs[2].set_ylabel('Flow (mol/min)', fontsize=16)
# axs[2].set_ylim((0,0.1))
# for item in (axs[2].get_xticklabels()):
#              item.set_fontsize(13)
# for item in (axs[2].get_yticklabels()):
#              item.set_fontsize(13)
# axs[2].set_xticks(np.arange(6,26,2))

# axs[2].set_xlim((5,25))  # cutting off where the horn paper cuts off the plots

# axs[2].set_xlabel('Position (mm)', fontsize=16)
# fig.set_figheight(10)
# fig.set_figwidth(7)
# plt.savefig('./paperplots/all_compare_subplotsPt_cov.pdf',bbox_inches='tight',dpi=300)
# plt.show()

In [20]:
dist = np.linspace(0,25,2501)


fig, axs = plt.subplots(3, 1)

# plotting ch4
axs[0].plot(dist_array_cov,gas_out_cov[:,3], '--',label=gas_names_cov[3],color='g')
axs[0].plot(dist_array,gas_out[:,3], ':',label=gas_names[3],color='g',)
axs[0].annotate("CH$_4$", fontsize=16, color='g',
            xy=(dist_array_cov[700], gas_out_cov[:, 3][500] + gas_out_cov[:, 3][500] / 100.0),
            va='bottom', ha='center')
axs[0].plot(dist,ch4(dist),color='g')

# plotting o2
axs[0].plot(dist_array,gas_out[:,4], '--',label=gas_names[4],color='orange')
axs[0].plot(dist_array_cov,gas_out_cov[:,4], ':',label=gas_names_cov[4],color='orange')
axs[0].annotate("O$_2$", fontsize=16, color='orange',
            xy=(dist_array[700], gas_out[:, 4][500] + gas_out[:, 4][500] / 100.0),
            va='bottom', ha='center')
axs[0].plot(dist,o2(dist),color='orange')

axs[0].plot([dist_array[on_catalyst], dist_array[on_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
axs[0].plot([dist_array[off_catalyst], dist_array[off_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
axs[0].annotate("catalyst", fontsize=13, xy=(dist_array[1500], 0.095), va='center', ha='center')

axs[0].set_ylabel('Flow (mol/min)', fontsize=16)
axs[0].set_ylim((0,0.1))
for item in (axs[0].get_xticklabels()):
             item.set_fontsize(13)
for item in (axs[0].get_yticklabels()):
             item.set_fontsize(13)
axs[0].set_xticks(np.arange(6,26,2))

ax2 = axs[0].twinx()
ax2.plot(dist,temp(dist), '--', color='r')
ax2.plot(dist_array,T_array,color='r')
ax2.annotate("T (K)",fontsize=16, color='r',
             xy=(dist_array[2300], T_array[2300] +150),
             va='top', ha='center')
ax2.set_ylim(300, 2000)
for item in (ax2.get_yticklabels()):
             item.set_fontsize(13)
axs[0].set_xlim((5,25))

# plotting h2
axs[1].plot(dist_array,gas_out[:,7], '--',label=gas_names[7],color='k',)
axs[1].plot(dist_array_cov,gas_out_cov[:,7], ':',label=gas_names_cov[7],color='k',)
axs[1].annotate("H$_2$", fontsize=16, color='k',
            xy=(dist_array[2200], 0.0575),
            va='bottom', ha='center')
axs[1].plot(dist,h2(dist),color='k')

# plotting co
axs[1].plot(dist_array,gas_out[:,8], '--', label=gas_names[8],color='limegreen')
axs[1].plot(dist_array_cov,gas_out_cov[:,8], ':',label=gas_names_cov[8],color='limegreen')
axs[1].annotate("CO", fontsize=16, color='limegreen',
            xy=(dist_array[1800], gas_out[:, 8][1800] + gas_out[:, 8][1800] / 100.0 - 0.1),
            va='bottom', ha='center')
axs[1].plot(dist,co(dist),color='limegreen')

axs[1].plot([dist_array[on_catalyst], dist_array[on_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
axs[1].plot([dist_array[off_catalyst], dist_array[off_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')

axs[1].set_ylabel('Flow (mol/min)', fontsize=16)
axs[1].set_ylim((0,0.1))
for item in (axs[1].get_xticklabels()):
             item.set_fontsize(13)
for item in (axs[1].get_yticklabels()):
             item.set_fontsize(13)
axs[1].set_xticks(np.arange(6,26,2))

axs[1].set_xlim((5,25))  # cutting off where the horn paper cuts off the plots

# plotting h2o
axs[2].plot(dist_array,gas_out[:,6], '--', label=gas_names[6],color='dodgerblue')
axs[2].plot(dist_array_cov,gas_out_cov[:,6], ':', label=gas_names_cov[6],color='dodgerblue')
axs[2].annotate("H$_2$O", fontsize=16, color='dodgerblue',
            xy=(dist_array[2200], 0.035),
            va='bottom', ha='center')
axs[2].plot(dist,h2o(dist),color='dodgerblue')

# plotting co2
axs[2].plot(dist_array,gas_out[:,5], '--', label=gas_names[5],color='blue')
axs[2].plot(dist_array_cov,gas_out_cov[:,5], ':', label=gas_names_cov[5],color='blue',)
axs[2].annotate("CO$_2$", fontsize=16, color='blue',
            xy=(dist_array[1400], .009),
            va='bottom', ha='center')
axs[2].plot(dist,co2(dist),color='blue')

axs[2].plot([dist_array[on_catalyst], dist_array[on_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')
axs[2].plot([dist_array[off_catalyst], dist_array[off_catalyst]], [0, 0.1], linestyle=':', color='xkcd:grey')

axs[2].set_ylabel('Flow (mol/min)', fontsize=16)
axs[2].set_ylim((0,0.1))
for item in (axs[2].get_xticklabels()):
             item.set_fontsize(13)
for item in (axs[2].get_yticklabels()):
             item.set_fontsize(13)
axs[2].set_xticks(np.arange(6,26,2))

axs[2].set_xlim((5,25))  # cutting off where the horn paper cuts off the plots

axs[2].set_xlabel('Position (mm)', fontsize=16)
fig.set_figheight(10)
fig.set_figwidth(7)
plt.savefig('./paperplots/all_compare_subplots.pdf',bbox_inches='tight',dpi=300)
# plt.show()